In [1]:
%run ../common.ipynb

In [2]:
pip install faker

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession
from faker import Faker
import random

# Initialize Spark Session with MySQL JDBC Driver
spark = SparkSession.builder.appName("load_operation_data") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.33") \
    .getOrCreate()


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


:: loading settings :: url = jar:file:/home/glue_user/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/glue_user/.ivy2/cache
The jars for the packages stored in: /home/glue_user/.ivy2/jars
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cb0e1003-315b-4ec3-baf7-1f94c7e6367e;1.0
	confs: [default]
	found mysql#mysql-connector-java;8.0.33 in central
	found com.mysql#mysql-connector-j;8.0.33 in central
	found com.google.protobuf#protobuf-java;3.21.9 in central
:: resolution report :: resolve 182ms :: artifacts dl 4ms
	:: modules in use:
	com.google.protobuf#protobuf-java;3.21.9 from central in [default]
	com.mysql#mysql-connector-j;8.0.33 from central in [default]
	mysql#mysql-connector-java;8.0.33 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-------------------------------------------------------------

In [4]:
# Initialize Faker for realistic data
fake = Faker()

# Generate Customers
def generate_customers(n):
    return spark.createDataFrame(
        [(i, fake.first_name(), fake.last_name(), fake.email(), fake.phone_number(),
          fake.address(), fake.city(), fake.state(), fake.zipcode(), fake.country()) 
         for i in range(1, n+1)], 
        ["customer_id", "first_name", "last_name", "email", "phone", "address", "city", "state", "zip_code", "country"]
    )

In [5]:
# Generate Data
customers_df = generate_customers(10000)
customers_df.show()

+-----------+----------+---------+--------------------+--------------------+--------------------+-------------------+--------------+--------+--------------------+
|customer_id|first_name|last_name|               email|               phone|             address|               city|         state|zip_code|             country|
+-----------+----------+---------+--------------------+--------------------+--------------------+-------------------+--------------+--------+--------------------+
|          1|   Charles|    Jones|rodneysullivan@ex...| (337)512-3234x99488|365 Ricky Spur\nK...|         Jacobmouth|        Kansas|   72281|United States of ...|
|          2|   Brandon|  Mcgrath|crystalnavarro@ex...|  953.750.7854x37299|6373 Susan Inlet ...|       Chambersview|      Oklahoma|   20216|          Luxembourg|
|          3|     Misty|   Warren| fpineda@example.com|   315.763.1867x5725|67991 Michael Roa...|         Sarahshire|      Illinois|   37114|         Netherlands|
|          4| Alexande

In [6]:
# Remove duplicate emails before writing to MySQL
customers_df = customers_df.dropDuplicates(["email"])

customers_df.write.jdbc(url=USER_MYSQL_URL, table="Customers", mode="append", properties=MYSQL_PROPERTIES)

print("✅ Data successfully written to MySQL!")

✅ Data successfully written to MySQL!


In [7]:





# Generate Products
def generate_products(n):
    categories = ["Electronics", "Clothing", "Home & Kitchen", "Books", "Toys", "Sports"]
    return spark.createDataFrame(
        [(i, fake.word().capitalize(), fake.sentence(nb_words=6), 
          random.choice(categories), round(random.uniform(5, 500), 2), random.randint(10, 1000)) 
         for i in range(1, n+1)], 
        ["product_id", "name", "description", "category", "price", "stock_quantity"]
    )




# Generate Payments
def generate_payments(n, max_order_id):
    return spark.createDataFrame(
        [(i, random.randint(1, max_order_id), fake.date_this_decade(), 
          random.choice(["Credit Card", "PayPal", "Gift Card", "Bank Transfer"]), 
          round(random.uniform(10, 500), 2), random.choice(["Success", "Failed", "Pending"])) 
         for i in range(1, n+1)], 
        ["payment_id", "order_id", "payment_date", "payment_method", "amount", "status"]
    )

# Generate Purchase History
def generate_purchase_history(n, max_customer_id, max_order_id):
    return spark.createDataFrame(
        [(i, random.randint(1, max_customer_id), random.randint(1, max_order_id), 
          fake.date_this_decade(), round(random.uniform(10, 500), 2)) 
         for i in range(1, n+1)], 
        ["history_id", "customer_id", "order_id", "purchase_date", "amount_spent"]
    )

# Generate Usage History
def generate_usage_history(n, max_customer_id, max_product_id):
    return spark.createDataFrame(
        [(i, random.randint(1, max_customer_id), random.randint(1, max_product_id), 
          fake.date_this_decade(), fake.sentence(nb_words=10), 
          random.choice(["Returned", "Not Returned"])) 
         for i in range(1, n+1)], 
        ["usage_id", "customer_id", "product_id", "usage_date", "feedback", "return_status"]
    )

# Generate Transactions
def generate_transactions(n, max_customer_id, max_order_id, max_payment_id):
    return spark.createDataFrame(
        [(i, random.randint(1, max_customer_id), random.randint(1, max_order_id), 
          random.randint(1, max_payment_id), fake.date_this_decade(), 
          round(random.uniform(10, 500), 2), random.choice(["Purchase", "Refund"]), 
          random.choice(["Completed", "Failed", "Pending"])) 
         for i in range(1, n+1)], 
        ["transaction_id", "customer_id", "order_id", "payment_id", "transaction_date", 
         "amount", "transaction_type", "status"]
    )




In [8]:
products_df = generate_products(1000)

products_df.write.jdbc(url=PRODUCT_MYSQL_URL, table="Products", mode="append", properties=MYSQL_PROPERTIES)


In [9]:

customers_df = spark.read.jdbc(url=USER_MYSQL_URL, table="Customers", properties=MYSQL_PROPERTIES)

valid_customer_ids = [row["customer_id"] for row in customers_df.select("customer_id").collect()]

# ✅ Generate Orders using only valid customer IDs
def generate_orders(n):
    return spark.createDataFrame(
        [(i, random.choice(valid_customer_ids), fake.date_this_decade(), 
          random.choice(["Pending", "Shipped", "Delivered", "Cancelled"]), 
          round(random.uniform(10, 500), 2)) 
         for i in range(1, n+1)], 
        ["order_id", "customer_id", "order_date", "status", "total_amount"]
    )

orders_df = generate_orders(100000)
orders_df.show()

+--------+-----------+----------+---------+------------+
|order_id|customer_id|order_date|   status|total_amount|
+--------+-----------+----------+---------+------------+
|       1|       3856|2024-06-25|  Pending|      100.13|
|       2|       3671|2020-08-07|Delivered|      101.84|
|       3|       5160|2024-08-05|Delivered|      476.69|
|       4|       6403|2020-06-08|Cancelled|      221.15|
|       5|       3469|2021-10-22|Cancelled|      390.76|
|       6|       2636|2022-04-08|  Pending|      214.12|
|       7|       3706|2023-01-15|Delivered|       69.41|
|       8|       6482|2021-01-26|Delivered|      452.21|
|       9|       3540|2024-04-21|  Pending|      222.33|
|      10|       2356|2024-01-01|Cancelled|      244.17|
|      11|        630|2023-11-14|Delivered|       308.8|
|      12|       2607|2023-04-21|  Pending|       37.98|
|      13|       5700|2021-06-27|Delivered|      140.21|
|      14|        494|2022-05-26|Cancelled|      325.17|
|      15|       9003|2021-11-0

In [10]:

orders_df.write.jdbc(url=ORDER_MYSQL_URL, table="Orders", mode="append", properties=MYSQL_PROPERTIES)

In [11]:

#orders_df.write.jdbc(url=MYSQL_URL, table="Orders", mode="append", properties=MYSQL_PROPERTIES)

orders_df = spark.read.jdbc(url=ORDER_MYSQL_URL, table="Orders", properties=MYSQL_PROPERTIES)
products_df = spark.read.jdbc(url=PRODUCT_MYSQL_URL, table="Products", properties=MYSQL_PROPERTIES)
valid_order_ids = [row["order_id"] for row in orders_df.select("order_id").collect()]
valid_product_ids = [row["product_id"] for row in products_df.select("product_id").collect()]
valid_customer_ids = [row["customer_id"] for row in customers_df.select("customer_id").collect()]
# ✅ Generate Order Items using only valid order_id & product_id
def generate_order_items(n):
    return spark.createDataFrame(
        [(i, random.choice(valid_order_ids), random.choice(valid_product_ids), 
          random.randint(1, 5), round(random.uniform(5, 500), 2)) 
         for i in range(1, n+1)], 
        ["order_item_id", "order_id", "product_id", "quantity", "unit_price"]
    )
order_items_df = generate_order_items(100000)
order_items_df.write.jdbc(url=ORDER_MYSQL_URL, table="Order_Items", mode="append", properties=MYSQL_PROPERTIES)


In [12]:
# Generate Payments
def generate_payments(n):
    return spark.createDataFrame(
        [(i,random.choice(valid_order_ids), fake.date_this_decade(), 
          random.choice(["Credit Card", "PayPal", "Gift Card", "Bank Transfer"]), 
          round(random.uniform(10, 500), 2), random.choice(["Success", "Failed", "Pending"])) 
         for i in range(1, n+1)], 
        ["payment_id", "order_id", "payment_date", "payment_method", "amount", "status"]
    )
payments_df = generate_payments(10000)
valid_payments_ids = [row["payment_id"] for row in payments_df.select("payment_id").collect()]

# Generate Purchase History
def generate_purchase_history(n):
    return spark.createDataFrame(
        [(i,random.choice(valid_customer_ids), random.choice(valid_order_ids), 
          fake.date_this_decade(), round(random.uniform(10, 500), 2)) 
         for i in range(1, n+1)], 
        ["history_id", "customer_id", "order_id", "purchase_date", "amount_spent"]
    )

payments_df.write.jdbc(url=ORDER_MYSQL_URL, table="Payments", mode="append", properties=MYSQL_PROPERTIES)
purchase_history_df = generate_purchase_history(1000)
purchase_history_df.distinct().write.jdbc(url=ORDER_MYSQL_URL, table="Purchase_History", mode="append", properties=MYSQL_PROPERTIES)


In [13]:

# Generate Usage History
def generate_usage_history(n):
    return spark.createDataFrame(
        [(i,random.choice(valid_customer_ids),random.choice(valid_product_ids),fake.date_this_decade(), fake.sentence(nb_words=10), 
          random.choice(["Returned", "Not Returned"])) 
         for i in range(1, n+1)], 
        ["usage_id", "customer_id", "product_id", "usage_date", "feedback", "return_status"]
    )

# Generate Transactions
def generate_transactions(n,):
    return spark.createDataFrame(
        [(i, random.choice(valid_customer_ids), random.choice(valid_order_ids), 
          random.choice(valid_payments_ids), fake.date_this_decade(), 
          round(random.uniform(10, 500), 2), random.choice(["Purchase", "Refund"]), 
          random.choice(["Completed", "Failed", "Pending"])) 
         for i in range(1, n+1)], 
        ["transaction_id", "customer_id", "order_id", "payment_id", "transaction_date", 
         "amount", "transaction_type", "status"]
    )
usage_history_df = generate_usage_history(1000)
usage_history_df.distinct().write.jdbc(url=ORDER_MYSQL_URL, table="Usage_History", mode="append", properties=MYSQL_PROPERTIES)

transactions_df = generate_transactions(1000)
transactions_df.distinct().write.jdbc(url=ORDER_MYSQL_URL, table="Transactions", mode="append", properties=MYSQL_PROPERTIES)

print("✅ Data successfully written to MySQL!")

✅ Data successfully written to MySQL!


In [ ]:
from pyspark.sql.functions import when,rand,lit
# Generate random login data
def generate_logins(customers_df, num_attempts=50):
    logins = []
    for row in customers_df.collect():
        customer_id = row["customer_id"]
        num_logins = random.randint(1, num_attempts)  # Random login attempts per customer
        
        for _ in range(num_logins):
            ip = f"192.168.{random.randint(0, 255)}.{random.randint(0, 255)}"
            login_attempt = random.choice(["SUCCESS", "FAILURE"])
            login_date = f"2024-02-{random.randint(1, 28)}"
            
            logins.append((customer_id, ip, login_attempt, login_date))
    
    return logins

# Generate synthetic login data
login_data = generate_logins(customers_df)

# Create DataFrame
columns = ["customer_id", "ip_address", "login_attempt", "login_date"]
login_df = spark.createDataFrame(login_data, columns)

# Introduce suspicious activity (customers with >3 IPs or >10 attempts)
login_df = login_df.withColumn("is_suspicious", when((rand() < 0.2), lit(1)).otherwise(lit(0)))

# Write to MySQL (optional)
login_df.write.jdbc(url=USER_MYSQL_URL, table="LoginHistory", mode="append", properties=MYSQL_PROPERTIES)

# Show result
login_df.show()